In [1]:
import igraph as ig

import numpy as np
import os
#Visualizations
import matplotlib.pyplot as plt
import random
from datetime import datetime
import itertools

import swnHeatKernels as swnN 
from scipy import linalg
import helperFunc as hf
import swnMetrics as swn
import centrality as ce
import distributions as dstr

from scipy.stats.stats import pearsonr 
from numpy.polynomial.polynomial import polyfit

## Functions

In [2]:
#calculates Qrandom and Q after rewirings
def getQRandomQFinal(adjDict):
    
    repetitions = len(adjDict)
    keyTuple = list(adjDict[1].keys())[0]
    rewirings = keyTuple[2]
    Q0 = np.zeros(repetitions)
    Q1 = np.zeros(repetitions)
    
    for ind, k in enumerate(adjDict):
        #load the specific rewired matrices -random and rewired
        A0 = adjDict[k][keyTuple][0]
        A1 = adjDict[k][keyTuple][1]
        
        #store them in the Q vectors
        Q0[ind] = getModularityScoreMultilevel(A0)
        Q1[ind] = getModularityScoreMultilevel(A1)
        
        
    Q = {}
    Q[0] = Q0
    Q[rewirings] = Q1
    
    return Q

In [3]:
def getModularityScoreMultilevel(A):
    
    #construct it so that igraph can read it, make it undirected
    g = ig.Graph(directed=False)
    #make it weighted
    g.es["weight"] = 1.0                
                
    g.add_vertices(len(A))
    ix, jx = A.nonzero()
    for i,j in zip(ix, jx):
        if i<j:
            g.add_edge(i, j, weight=A[i,j])
            
    #calculate the clusters and their modularity score    
    clusters = g.community_multilevel(weights=g.es['weight'])
    modularity_score = g.modularity(clusters.membership,weights=g.es['weight'])
    
    return modularity_score

## Load A compute and save Q

In [4]:
rewirings = 4000
pRand = [0, 0.2]

weightDist = ['normal','lognormal']
directoryLoad ='data/ArandA/1000iterations/QrandQtau/'
directoryQSave ='data/ModularityValues/1000iterations/QrandQtau/'
description = ['Mod','Trans','Centra']

In [6]:

dataA = {}; Q = {}; taus = {}
for descr in description:
    for wD in weightDist:
        for indP, p in enumerate(pRand):       
            #Load A
            filePathLoad = directoryLoad + 'ArandA_tau'+descr+'_'+wD+'_p'+str(p)+'_rewir'+str(rewirings)+'.pckl'
            dataA[(wD,p)] = hf.loadVar(filePathLoad)
            taus[(wD,p)] = list(dataA[(wD,p)][1].keys())[0][1]
            #calculate Q
            Q[(wD,p)] = getQRandomQFinal(dataA[(wD,p)])

            filePathQSave = directoryQSave + descr+'.pckl'
            print(filePathQSave)
            hf.saveVarSimple((Q,taus), filePathQSave)        


data/ModularityValues/1000iterations/QrandQtau/Mod.pckl
data/ModularityValues/1000iterations/QrandQtau/Mod.pckl
data/ModularityValues/1000iterations/QrandQtau/Mod.pckl
data/ModularityValues/1000iterations/QrandQtau/Mod.pckl
data/ModularityValues/1000iterations/QrandQtau/Trans.pckl
data/ModularityValues/1000iterations/QrandQtau/Trans.pckl
data/ModularityValues/1000iterations/QrandQtau/Trans.pckl
data/ModularityValues/1000iterations/QrandQtau/Trans.pckl
data/ModularityValues/1000iterations/QrandQtau/Centra.pckl
data/ModularityValues/1000iterations/QrandQtau/Centra.pckl
data/ModularityValues/1000iterations/QrandQtau/Centra.pckl
data/ModularityValues/1000iterations/QrandQtau/Centra.pckl
